# Implementation of 

## <center>  PseudoEdgeNet: Nuclei Segemtnation only with Point Annotations

## <center> Questions

* kernel size of PseudoEdgeNet

* 

## <center> Training Data

### Info

**TCGA data set:**

* 30 H&E stained whole slide images from TCGA, have 7 organs, one WSI per patient, come from 18 hospitals.

* Then crop 30 sub-images, each size is 1000 * 1000, and keep only one cropped image per WSI and patient

* H&E stained, captured at 40X magnification

**TASK:**

* Instance segmentation

### Annotation

* Pixel coordinates of annoated nuclear boundaries

* For over-lapping nuclei, assign multi-nuclear pixel to the largest nucleus containing that pixel

* How to deal with float coordinates?

    * round them, cast to int

* source:

    * https://stackoverflow.com/questions/10735817/how-to-deal-with-floating-point-coordinate-values-in-image

    * https://stackoverflow.com/questions/58991754/how-to-use-float-number-as-opencv-image-pixel-coordinates



## <center> Preprocessing

### Generate Point Annotation(How to find centroid of any shape)

**source:**

* https://stackoverflow.com/questions/23020659/fastest-way-to-calculate-the-centroid-of-a-set-of-coordinate-tuples-in-python-wi

* https://www.learnopencv.com/find-center-of-blob-centroid-using-opencv-cpp-python/

* https://www.pyimagesearch.com/2016/02/01/opencv-center-of-contour/

**centroid definition in paper**:

* center of mass of each nucleus instatnce mask

**centroid calculation:**

* centroid = $\frac{\sum_{i=0}^n x_i}{n}$, centroid of any shape is the average of contour coordinates, where $x_i$ is the coordinate, n is the length of contour or number of pixel of contour in here.

**Contour structure in Opencv**:

* Contour is a list, length is the number of nuclei in image

* for each element, it is a contour of a single blob, each element is a numpy array has this shape: (number of pixel of contour, 1, 2)

* for each pixel of contour, it is a numpy array with shape of (1, 2)

**In my implementation:**

* construct contour data structure by groud truth, then use `cv.moment()` find centroid

### Generate voronoi boundary

* use opencv `cv.subdiv()` to compute voronoi boundary, then use `cv.polylines()` to assign value

## <center> Architecture

### Input:

* image I

### Loss Function:

* old: $\mathcal{L}(I, P, f, g) = \mathcal{L}_{ce}(f(I), P) + \lambda \cdot |s(f(I)) - g(I)|$

* new: $\mathcal{L}(I, P, f, g) = \mathcal{L}_{ce}(f(I), P) + \lambda \cdot |s(f(I)) - g(I) \otimes h(I)|$

### $\mathcal{L}_{ce}$:

* pixel-averaged cross-entropy loss

* positive labels**(with weight 1.0)** are given to the pixels corresponding to point annotations,

* negative labels**(with weight 0.1)** are assigned to pixels on Voronoi boundaries that can be obtained by distance transform with point annotations

* ignore all pixel that are not annotated

### $f(I)$:

* employ a Feature Pyramid Network 

* with ResNet-50 backbone followed by a sigmoid layer as the segmentation network

* threshold to determine positive pixels from f(I) is 0.5

* more detail: http://presentations.cocodataset.org/COCO17-Stuff-FAIR.pdf

### P:

* point annotation

### $\lambda$:

* scaling constant 1.0

### $s()$:

* (x, y) directional Sobel filter

### $g(I)$:

* EdgeNet to detect edge, Jointly learn in training time, not used in inference time

* Structure:

    * 4 CNN layer with 64 filters followed by batch normalization and ReLU
    
    * kernel size ? 3 * 3?

    * except for the last layer, produces a two-channel output represent (x, y)-directional Sobel edge maps, one for x, another for y

* small g is good, with 2,4,6,8 conv layer 

### $h(I)$:

* use FPN with a Resnet-18 backbone

* stack a sigmoid as an output layer

* $\otimes$ represent element-wise multiplication

## <center> f--segmentation Network

### Backbone

**ResNet50:**

* [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385)(Original)

* [Identity Mappings in Deep Residual Networks](https://arxiv.org/abs/1603.05027)(improved)

* I thought pre-trained network have no help to nuclei segmentation, therefore, pre-trained ResNet50 is not used

* original paper use initial version of ResNet

**FPN:**

* In decode process, add a crop layer after each upsampling layer to maintain output shape.

* crop left column and top row 

* source: https://stackoverflow.com/questions/46035581/how-to-ensure-caffe-segmentation-network-output-size-is-the-same-as-input

## <center> data augmentation

* Color jittering
    * https://stackoverflow.com/questions/35152636/random-flipping-and-rgb-jittering-slight-value-change-of-image

* Gaussian blurring

* Gaussian noise injection
    * https://answers.opencv.org/question/79758/how-to-add-gaussian-noise-in-all-channels-of-an-image-with-randn/

* rotation
    * https://docs.opencv.org/master/dd/d52/tutorial_js_geometric_transformations.html
* horizontal filp
    * https://docs.opencv.org/master/d2/de8/group__core__array.html#gaca7be533e3dac7feb70fc60635adf441
    
* vertical flip
    * https://docs.opencv.org/master/d2/de8/group__core__array.html#gaca7be533e3dac7feb70fc60635adf441
* affine transformation

* elastic deformation

## <center> Implementation

### Optimizer:

* Adam, with an initial learning rate of 0.001,

* learning rate is halved when the average loss per epoch does not decrease for current 5 epochs

* patience = 4

* plateau:

    * https://www.deeplearningwizard.com/deep_learning/boosting_models_pytorch/lr_scheduling/#reduce-on-loss-plateau-decay-patience0-factor01

    * https://pytorch.org/docs/stable/optim.html

### Evalutaion metric:

* Intersection over union (IoU)
    * https://www.kaggle.com/c/data-science-bowl-2018/discussion/51553
    * https://www.kaggle.com/c/tgs-salt-identification-challenge/discussion/63044

## <center> First trial

### Implementation

* with PseudoEdgeNet, without attention Module

* with all augmentation except affine and elastic deformation

* batch_size = 4

* epoch = 20